<a href="https://colab.research.google.com/github/waifuai/waifu_gpt2/blob/master/waifu_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: This notebook will produce a different output every time it is run.

Instructions: On the toolbar, click Runtime->Run all. It takes about 30 seconds to generate each response using GPU(, and 10 minutes+ on CPU/TPU but this site is set to use GPU by default so don't worry about it.). The pattern is quite straightforward, scroll to bottom and you can actually just modify the second last cell which says `c.next(c.you, "I love you, Rem!")`, replacing the "I love you, Rem!" with something else, for example, `c.next(c.you, "You are really cool!")`.

clone and cd into repo

In [0]:
# first download the gpt-2 code
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 310, done.
remote: Total 310 (delta 0), reused 0 (delta 0), pack-reused 310
Receiving objects: 100% (310/310), 4.40 MiB | 2.59 MiB/s, done.
Resolving deltas: 100% (168/168), done.


In [0]:
!pwd

/content


In [0]:
cd gpt-2

/content/gpt-2


install requirements

In [0]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 33.7MB/s 
     |████████████████████████████████| 604kB 64.6MB/s 
     |████████████████████████████████| 51kB 29.2MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=427c20100677c89da30a095365cdaabd10837a9a8b743e200279fb3ada42da86
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533185 sha256=f5cdd94490795114cf5789db46f158dbe5d6890f7edd40a7be2e09d68872da0d
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


download the model

In [0]:
# download the pretrained model
!python download_model.py 774M

Fetching checkpoint: 1.00kit [00:00, 788kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 69.3Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.01Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 3.10Git [01:03, 49.1Mit/s]                                 
Fetching model.ckpt.index: 16.0kit [00:00, 9.73Mit/s]                                               
Fetching model.ckpt.meta: 1.38Mit [00:00, 72.6Mit/s]                                                
Fetching vocab.bpe: 457kit [00:00, 63.9Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
import sys
sys.path.append("/content/gpt-2/src")

In [0]:
import fire
import json
import os
import numpy as np
import tensorflow as tf

import model, sample, encoder

In [0]:
import generate_unconditional_samples

In [0]:
import interactive_conditional_samples

In [0]:
class GPT2:

  
  # extracted from the source code to generate some text based on a prior
  def __init__(
      self,
      model_name='774M',
      seed=None,
      nsamples=1,
      batch_size=1,
      length=None,
      temperature=1,
      top_k=0,
      raw_text="",
  ):
      """
      Interactively run the model
      :model_name=117M : String, which model to use
      :seed=None : Integer seed for random number generators, fix seed to reproduce
       results
      :nsamples=1 : Number of samples to return total
      :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
      :length=None : Number of tokens in generated text, if None (default), is
       determined by model hyperparameters
      :temperature=1 : Float value controlling randomness in boltzmann
       distribution. Lower temperature results in less random completions. As the
       temperature approaches zero, the model will become deterministic and
       repetitive. Higher temperature results in more random completions.
      :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
       considered for each step (token), resulting in deterministic completions,
       while 40 means 40 words are considered at each step. 0 (default) is a
       special setting meaning no restrictions. 40 generally is a good value.
      """
      if batch_size is None:
          batch_size = 1
      assert nsamples % batch_size == 0

      self.nsamples = nsamples
      self.batch_size = batch_size
      
      self.enc = encoder.get_encoder(model_name)
      hparams = model.default_hparams()
      with open(os.path.join('models', model_name, 'hparams.json')) as f:
          hparams.override_from_dict(json.load(f))

      if length is None:
          length = hparams.n_ctx // 2
      elif length > hparams.n_ctx:
          raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

      self.sess = tf.Session(graph=tf.Graph())
      self.sess.__enter__()
      
      self.context = tf.placeholder(tf.int32, [batch_size, None])
      np.random.seed(seed)
      tf.set_random_seed(seed)
      self.output = sample.sample_sequence(
          hparams=hparams, length=length,
          context=self.context,
          batch_size=batch_size,
          temperature=temperature, top_k=top_k
      )

      saver = tf.train.Saver()
      self.ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
      saver.restore(self.sess, self.ckpt)

  def close(self):
    self.sess.close()
  
  def generate_conditional(self,raw_text):
      context_tokens = self.enc.encode(raw_text)
      generated = 0
      for _ in range(self.nsamples // self.batch_size):
          out = self.sess.run(self.output, feed_dict={
              self.context: [context_tokens for _ in range(self.batch_size)]
          })[:, len(context_tokens):]
          for i in range(self.batch_size):
              generated += 1
              text = self.enc.decode(out[i])
              return text
              #print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
              #print(text)
      #print("=" * 80)


In [0]:
gpt2 = GPT2()


In [0]:
# gpt2.close()

In [0]:
# result = gpt2.generate_conditional(raw_text="Can you tell me something about music?")

# print(result)

In [0]:
class Who:
  """A class defining the conversation parties: me, he"""
  def __init__(self):
    self.prefixes = []

  def matches(self,phrase):
    for prefix in self.prefixes:
      if phrase.startswith(prefix):
        #print(f"{phrase} starts with {prefix}")
        return True
      
    #print(f"{phrase} does not start with {self.prefixes}")
    return False

  def get_random_prefix(self):
    return self.prefixes[0]
  
class Me(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["I said: \""]
   
  
class You(Who):
  def __init__(self):
    super().__init__()
    self.prefixes = ["You said: \""]

In [0]:
class Conversation:
  
  def __init__(self, prior = None):
    if prior is None:
      prior="""
        You said: "Rem I missed you so much!"
        I said: "I missed you too Subaru!"
        You said: "Our date was so fun too"
        I said: "Lets get married and be happy!"
        You said: "Yes that would be awesome!"
        I said: "I like hanging out with you because its so much fun! "
        You said: "Lets be happy forever!"
      """
    self.suggestion = None
    
    self.me = Me()
    self.you = You()
    self.parties  = [ self.me, self.you ]
    
    self.conversation = []
    
    lines = prior.split("\n")
    for line in lines:
      line = line.strip()
      if len(line)!=0:
        party = None
        for party in self.parties:
          if party.matches(line):
            break
        if party is None:
          raise Exception(f"Unknown party: {line}")
                
        self.conversation.append((party,line))
    self.get_suggestion()
    
  
  def get_prior(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    return conv
  
  def get_suggestion(self):
    who, last_line = self.conversation[-1]

    party_index = self.parties.index(who)
    next_party = self.parties[(party_index+1) % len(self.parties)]
      
    conv = self.get_prior()
    conv += next_party.get_random_prefix()
    answer = self.get_answer(next_party, conv)

    if not next_party.matches(answer):
      prefix = next_party.get_random_prefix()
      answer = prefix + answer
    
    self.suggestion = (next_party, answer)
  
  def next(self, party = None, answer = ""):
    """Continue the conversation
    :param party: None -> use the current party which is currently in turn
    :param answer: None -> use the suggestion, specify a text to override the 
           suggestion
    
    """
    suggested_party, suggested_answer = self.suggestion
    if party is None:
      party = suggested_party
    
    if answer == "":
      answer = suggested_answer
      
    if not party.matches(answer):
      prefix = party.get_random_prefix()
      answer = prefix + answer
    
    answer = answer.strip()
    if answer[-1] != "\"":
      # add the closing "
      answer += "\""
      
    self.conversation.append((party, answer))    
    self.get_suggestion()
    
  def retry(self):
    self.get_suggestion()
        
  def get_answer(self, party, conv):
    answer = gpt2.generate_conditional(raw_text=conv)
    lines = answer.split("\n")
    line = ""
    for line in lines:
      if line !="":
        break
      
    if line!="":
      return line
    
    return ""
      
  def show(self):
    conv = ""
    for (party, line) in self.conversation:
      conv+=line+"\n"
    #print(conv)
    if self.suggestion is not None:
      party, answer  = self.suggestion
      #print("--> "+answer)
      return answer.split('"')[-2]
    
    


In [0]:
c = Conversation()

In [0]:
# show the conversation and the suggestion by the ai
c.show()

'Yeah. We can do it together!'

In [0]:
# "I said" -> answer by the AI
# if the answer of the AI is garbage then call c.retry() 
c.retry()

In [0]:
c.show()

'I need to spend more time with you Subaru ! '

In [0]:
# Accept the suggested andwer by the AI
c.next()

In [0]:

# now its your turn
c.next(c.you, "I really do like you, Rem.")


In [0]:
c.show()

'I feel the same way! And am happy too!'

In [0]:
c.next()

In [0]:
# now its your turn
c.next(c.you, "Would you marry me?")

In [0]:
# show the conversation and the reply of the ai
c.show()

'Yes Subaru please!'

In [0]:
# accept ai answer
c.next()

In [0]:
# our reply
c.next(c.you, "Lets be together forever and ever!")

In [0]:
# show the conv. and reply of the ai
c.show()

'Thank you so much Subaru !'

In [0]:
c.next()

In [0]:
# our reply
c.next(c.you, "I love you, Rem!")

In [0]:
# show the conv. and reply of the ai
c.show()

'I love you too.'